# 模型選擇與參數調整 (Model Selection and Parameter Tuning)

這個 Notebook 將介紹如何進行模型選擇和參數調整，包括以下方法：

1.  **GridSearchCV**
2.  **RandomizedSearchCV**
3.  **Bayesian Optimization**

我們將使用 `pandas`、`numpy` 和 `scikit-learn` 等套件來實作這些方法，並使用 `bayesian-optimization` 套件進行貝葉斯優化，並以鳶尾花資料集和隨機森林分類器為例進行說明。

## 載入需要的套件

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from bayes_opt import BayesianOptimization
# from skopt import BayesSearchCV

## 安裝 `bayesian-optimization` 套件

如果尚未安裝 `bayesian-optimization` 套件，可以使用以下指令安裝：

```bash
pip install bayesian-optimization
```

如果需要使用基於 `scikit-optimize` 的 `BayesSearchCV`，可以使用以下命令安裝 `scikit-optimize`：

```bash
pip install scikit-optimize
```

---

## 資料準備

In [ ]:
# 載入鳶尾花資料集
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

# 將資料劃分為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

---

## 模型選擇與參數調整的重要性

在機器學習中，**模型選擇**和**參數調整**是至關重要的步驟。不同的模型有不同的特性和適用場景，而同一個模型也可能因為參數的不同而產生截然不同的效能。

**模型選擇:** 指的是根據問題的性質和資料的特性，從多個候選模型中選擇一個最合適的模型。

**參數調整:** 指的是對選定的模型，調整其超參數 (Hyperparameters) 以獲得最佳效能。超參數是在模型訓練之前設定的參數，而不是通過訓練資料學習得到的。

**目標:**  找到最佳的模型和參數組合，使得模型在未見過的資料上也能表現良好 (即具有良好的泛化能力)。

**常用的方法:**

-   **GridSearchCV:**  窮舉搜尋所有指定的參數組合，並使用交叉驗證評估每個組合的效能。缺點是當參數空間很大時，計算成本很高。
-   **RandomizedSearchCV:**  從指定的參數分佈中隨機採樣參數組合，並使用交叉驗證評估每個組合的效能。優點是可以更快地探索更大的參數空間，缺點是可能無法找到最佳的參數組合。
-   **Bayesian Optimization:**  使用機率模型來預測參數組合的效能，並根據預測結果選擇下一個要評估的參數組合。優點是可以更有效地找到最佳的參數組合，缺點是實現起來較為複雜。 (`BayesSearchCV` 可以參考但較為不推薦)

接下來，我們將分別示範如何使用這三種方法來進行模型選擇和參數調整。

---

<a id='gridsearchcv'></a>
## 1. GridSearchCV

`GridSearchCV` 是一種常用的參數調整方法，它會嘗試所有指定的參數組合，並使用交叉驗證來評估每個組合的效能。最後，它會返回最佳的參數組合和對應的模型。

**優點:**

-   可以找到指定參數空間中的最佳參數組合。
-   實現簡單。

**缺點:**

-   計算成本高，特別是當參數空間很大時。
-   只能搜尋離散的參數值。

In [ ]:
# 定義參數空間
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 建立隨機森林分類器
rf = RandomForestClassifier(random_state=42)

# 建立 GridSearchCV 物件
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)

# 訓練模型並進行參數搜尋
grid_search.fit(X_train, y_train)

# 顯示最佳參數組合
print("Best parameters found: ", grid_search.best_params_)

# 顯示最佳模型
print("Best estimator found: ", grid_search.best_estimator_)

# 使用最佳模型預測測試集
y_pred = grid_search.predict(X_test)

# 評估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

---

<a id='randomizedsearchcv'></a>
## 2. RandomizedSearchCV

`RandomizedSearchCV` 是一種隨機搜尋參數的方法，它從指定的參數分佈中隨機採樣參數組合，並使用交叉驗證來評估每個組合的效能。與 `GridSearchCV` 相比，`RandomizedSearchCV` 可以更快地探索更大的參數空間。

**優點:**

-   可以更快地探索更大的參數空間。
-   可以指定連續型的參數分佈。

**缺點:**

-   可能無法找到最佳的參數組合。

In [ ]:
# 定義參數分佈
param_dist = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# 建立隨機森林分類器
rf = RandomForestClassifier(random_state=42)

# 建立 RandomizedSearchCV 物件
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=10, cv=3, scoring='accuracy', random_state=42, n_jobs=-1) # 調整 n_iter

# 訓練模型並進行參數搜尋
random_search.fit(X_train, y_train)

# 顯示最佳參數組合
print("Best parameters found: ", random_search.best_params_)

# 顯示最佳模型
print("Best estimator found: ", random_search.best_estimator_)

# 使用最佳模型預測測試集
y_pred = random_search.predict(X_test)

# 評估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

---

<a id='bayesian-optimization'></a>
## 3. Bayesian Optimization

貝葉斯優化是一種用於優化黑盒函數的全域優化方法，它通過構建目標函數的機率模型 (通常是高斯過程)，並利用這個模型來選擇下一個最有希望的評估點。貝葉斯優化通常比網格搜尋和隨機搜尋更有效，特別是在處理高維參數空間和計算成本昂貴的目標函數時。

**優點:**

-   通常比 `GridSearchCV` 和 `RandomizedSearchCV` 更有效地找到最佳參數。
-   適用於計算成本昂貴的目標函數。

**缺點:**

-   實現起來較為複雜。
-   需要選擇合適的機率模型和採集函數。

In [ ]:
# 定義目標函數 (要最大化的函數)
def objective_function(n_estimators, max_depth, min_samples_split, min_samples_leaf):
    rf = RandomForestClassifier(
        n_estimators=int(n_estimators),
        max_depth=int(max_depth),
        min_samples_split=int(min_samples_split),
        min_samples_leaf=int(min_samples_leaf),
        random_state=42
    )
    # 使用 3 折交叉驗證計算平均準確度
    score = cross_val_score(rf, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

# 定義參數空間
pbounds = {
    'n_estimators': (50, 150),
    'max_depth': (1, 30),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 4)
}

# 建立 BayesianOptimization 物件
optimizer = BayesianOptimization(f=objective_function, pbounds=pbounds, random_state=42)

# 進行優化
optimizer.maximize(init_points=5, n_iter=10)  # 調整 init_points 和 n_iter

# 顯示最佳參數組合
print("Best parameters found: ", optimizer.max['params'])

# 顯示最佳分數
print("Best score found: ", optimizer.max['target'])

In [ ]:
# 使用最佳參數建立模型
best_params = optimizer.max['params']
rf_best = RandomForestClassifier(
    n_estimators=int(best_params['n_estimators']),
    max_depth=int(best_params['max_depth']),
    min_samples_split=int(best_params['min_samples_split']),
    min_samples_leaf=int(best_params['min_samples_leaf']),
    random_state=42
)

# 訓練模型
rf_best.fit(X_train, y_train)

# 預測測試集
y_pred = rf_best.predict(X_test)

# 評估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

## (選用) 使用 `skopt` 的 `BayesSearchCV` 進行貝葉斯優化

`BayesSearchCV` 是 `scikit-optimize` 套件提供的一個類似於 `GridSearchCV` 和 `RandomizedSearchCV` 的工具，它使用貝葉斯優化來搜尋最佳的參數組合。`BayesSearchCV` 的優點是可以更有效地找到最佳參數，特別是在處理高維參數空間和計算成本昂貴的目標函數時。

**注意:** `BayesSearchCV` 的 API 和 `GridSearchCV`、`RandomizedSearchCV` 類似，但需要指定參數的搜尋空間 (search spaces)，而不是參數的網格或分佈。

**安裝 `scikit-optimize`:**
```bash
pip install scikit-optimize
```

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

# 定義參數搜尋空間
search_spaces = {
    'n_estimators': Integer(50, 150),
    'max_depth': Integer(1, 30),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 4),
    'bootstrap': Categorical([True, False])
}

# 建立隨機森林分類器
rf = RandomForestClassifier(random_state=42)

# 建立 BayesSearchCV 物件
bayes_search = BayesSearchCV(estimator=rf, search_spaces=search_spaces, n_iter=15, cv=3, scoring='accuracy', random_state=42, n_jobs=-1) # 調整 n_iter

# 進行優化
bayes_search.fit(X_train, y_train)

# 顯示最佳參數組合
print("Best parameters found: ", bayes_search.best_params_)

# 顯示最佳模型
print("Best estimator found: ", bayes_search.best_estimator_)

# 使用最佳模型預測測試集
y_pred = bayes_search.predict(X_test)

# 評估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

---

## 總結

這個 Notebook 介紹了三種常用的模型選擇和參數調整方法：`GridSearchCV`、`RandomizedSearchCV` 和貝葉斯優化 (使用 `bayesian-optimization` 套件 和 `skopt` 套件的 `BayesSearchCV`)。這些方法可以幫助我們找到最佳的模型和參數組合，以提高模型的效能。

**如何選擇合適的方法?**

-   **`GridSearchCV`:** 適用於參數空間較小且計算資源充足的情況。
-   **`RandomizedSearchCV`:** 適用於參數空間較大且計算資源有限的情況，或者需要快速探索參數空間的情況。
-   **貝葉斯優化:** 適用於計算成本昂貴的目標函數，或者需要更有效地找到最佳參數的情況。`bayesian-optimization`套件提供了更直覺的自定義優化方式，`BayesSearchCV` 則更貼近 `scikit-learn` 的使用習慣。

建議您根據具體的應用場景和需求來選擇合適的方法。在實際應用中，您可能需要嘗試多種方法，並比較它們的效能，才能找到最佳的模型和參數組合。

## 參考資料

-   [Scikit-learn Documentation: Tuning the hyper-parameters of an estimator](https://scikit-learn.org/stable/modules/grid_search.html)
-   [bayesian-optimization Documentation](https://github.com/fmfn/BayesianOptimization)
-   [scikit-optimize Documentation](https://scikit-optimize.github.io/)